In [1]:
import os

import numpy as np
import xarray as xr
import xesmf as xe

## ISIMIP

In [3]:
# Set path
# UPDATE THIS FOR REPRODUCTION
path = (
    "/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/isimip3b/"  # location of ISIMIP metrics
)

In [4]:
# Models
from utils import isimip_ssp_dict

models = list(isimip_ssp_dict.keys())

In [5]:
# All metrics are regridded to NEX-GDDP grid (same as CIL)
out_grid = xr.open_dataset(
    "/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/nex-gddp/avg/CanESM5.nc"
)

out_grid = xr.Dataset({"lat": out_grid.lat, "lon": out_grid.lon})

In [6]:
# Regridding function
def regrid(model, out_grid, out_path, metric):
    # Read native grid
    ds = xr.open_dataset(out_path + "native_grid/" + metric + "/" + model + ".nc")

    # Conservative regridder from xESMF
    cons_regridder = xe.Regridder(ds, out_grid, "conservative", periodic=True)
    ds_cons = cons_regridder(ds)

    # Store
    ds_cons.to_netcdf(out_path + "regridded/conservative/" + metric + "/" + model + ".nc")

In [7]:
%%time
# All metrics
for metric in ["avg", "max", "dry", "wet", "max5d", "hotdry"]:
    for model in models:
        # Check if already exists
        if os.path.isfile(path + "regridded/conservative/" + metric + "/" + model + ".nc"):
            # print(model + ' ' + metric + ' already done')
            continue

        # If not, regrid
        regrid(model, out_grid, path, metric)
        print(model + " " + metric)

# 'hot' slightly different
metric = "hot"
for var_id in ["tasmin", "tasmax", "tas"]:
    for model in models:
        model_var = model + "_" + var_id
        # Check if already exists
        if os.path.isfile(path + "regridded/conservative/" + metric + "/" + model_var + ".nc"):
            # print(model + ' ' + metric + ' ' + var_id + ' already done')
            continue

        # If not, regrid
        print(model + " " + metric + " " + var_id)
        regrid(model_var, out_grid, path, metric)

CanESM5 max5d
CNRM-CM6-1 max5d
CNRM-ESM2-1 max5d
EC-Earth3 max5d
GFDL-ESM4 max5d
IPSL-CM6A-LR max5d
MIROC6 max5d
MPI-ESM1-2-HR max5d
MRI-ESM2-0 max5d
UKESM1-0-LL max5d
CPU times: user 9min 9s, sys: 1min 2s, total: 10min 11s
Wall time: 10min 31s


## GARD-SV (carbonplan)

In [2]:
# Set paths
# UPDATE THIS FOR REPRODUCTION
path = "/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/carbonplan/"  # location of carbonplan metrics

In [3]:
# Models
from utils import gardsv_ssp_dict, gardsv_var_dict

models = list(gardsv_ssp_dict.keys())

In [4]:
# All metrics are regridded to NEX-GDDP grid (same as CIL)
out_grid = xr.open_dataset(
    "/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/nex-gddp/avg/CanESM5.nc"
)

out_grid = xr.Dataset({"lat": out_grid.lat, "lon": out_grid.lon})

In [6]:
# Regridding function
def regrid(model, out_grid, path, metric, var_id):
    # Read native grid
    ds = xr.open_dataset(path + "native_grid/GARD-SV/" + metric + "/" + model, engine="zarr")

    if var_id:
        ds = ds[[var for var in list(ds.data_vars) if var.split("_")[0] == var_id]]

    # Conservative regridder from xESMF
    cons_regridder = xe.Regridder(ds, out_grid, "conservative", periodic=True)
    ds_cons = cons_regridder(ds)

    # Store
    if var_id:
        ds_cons.to_netcdf(path + "regridded/conservative/GARD-SV/" + metric + "/" + model + "_" + var_id + ".nc")
    else:
        ds_cons.to_netcdf(path + "regridded/conservative/GARD-SV/" + metric + "/" + model + ".nc")

In [8]:
%%time
# All metrics
# WARNING: requires around 70GB RAM for hot and wet metrics
for metric in ["avg", "max", "dry", "wet", "max5d", "hotdry"]:
    for model in models:
        # Some models missing pr
        if (metric in ["dry", "wet", "hotdry"]) and ("pr" in gardsv_var_dict[model]):
            # Check if already exists
            if os.path.isfile(path + "regridded/conservative/GARD-SV/" + metric + "/" + model + ".nc"):
                continue

            # Regrid
            regrid(model, out_grid, path, metric, False)
            print(model + " " + metric)

        elif metric not in ["dry", "wet", "hotdry"]:
            # Check if already exists
            if os.path.isfile(path + "regridded/conservative/GARD-SV/" + metric + "/" + model + ".nc"):
                continue

            # Regrid
            regrid(model, out_grid, path, metric, False)
            print(model + " " + metric)

/storage/work/d/dcl5300/ENVS/micromamba/envs/climate-stack-mamba-2023-01/lib/python3.10/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


BCC-CSM2-MR max5d


/storage/work/d/dcl5300/ENVS/micromamba/envs/climate-stack-mamba-2023-01/lib/python3.10/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


MIROC6 max5d
CPU times: user 3min 33s, sys: 16.6 s, total: 3min 50s
Wall time: 4min


In [15]:
# Need to split up hot metric for RAM
metric = "hot"

for model in models:
    for var_id in ["tas", "tasmin", "tasmax"]:
        # Check if already exists
        if os.path.isfile(path + "regridded/conservative/GARD-SV/" + metric + "/" + model + "_" + var_id + ".nc"):
            continue

        # Regrid
        regrid(model, out_grid, path, metric, var_id)
        print(model + " " + metric)

/storage/work/d/dcl5300/ENVS/micromamba/envs/climate-stack-mamba-2022-11-1/lib/python3.10/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


BCC-CSM2-MR hot


/storage/work/d/dcl5300/ENVS/micromamba/envs/climate-stack-mamba-2022-11-1/lib/python3.10/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


BCC-CSM2-MR hot


/storage/work/d/dcl5300/ENVS/micromamba/envs/climate-stack-mamba-2022-11-1/lib/python3.10/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


...

MPI-ESM1-2-HR hot


/storage/work/d/dcl5300/ENVS/micromamba/envs/climate-stack-mamba-2022-11-1/lib/python3.10/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


MPI-ESM1-2-HR hot


/storage/work/d/dcl5300/ENVS/micromamba/envs/climate-stack-mamba-2022-11-1/lib/python3.10/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


MPI-ESM1-2-HR hot
